In [1]:
class Food:
    def __init__(self, rank, title, sel_price):
        self.rank = rank
        self.title = title
        self.sel_price = sel_price

    def __str__(self):
        return f"{self.rank}, {self.title}, {self.sel_price}"
    
    def to_dict(self):
        return {'rank':self.rank, 
                'title':self.title,
                'sel_price':self.sel_price
                }
    
    def to_list(self):
        return [self.rank, 
                self.title,
                self.sel_price]

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.gmarket.co.kr/n/best'
res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(res.content, 'html.parser')
best_list_el = soup.select('.list-item a.link')
len(best_list_el)

0

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Chrome 옵션 설정
options = Options()
# options.add_argument('--headless')  # 개발 중엔 꺼두는 게 좋아
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# G마켓 베스트 페이지 접속
driver.get("https://www.gmarket.co.kr/n/best")

# 로딩 대기
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "li.list-item"))
    )
    time.sleep(1)
except:
    print("상품이 로딩되지 않았습니다.")

# HTML 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 랭킹 있는 상품만 추출
valid_items = []

for item in soup.select("li.list-item"):
    if item.select_one('.box__label-rank'):  # 랭킹 태그 있는 애만
        valid_items.append(item)

# 결과 출력
print(f"\n실제 랭킹 상품 {len(valid_items)}개\n")

for i, item in enumerate(valid_items, start=1):
    title_tag = item.select_one('.box__item-title')
    price_tag = item.select_one('.box__price-seller .text__value')

    title = title_tag.text.strip() if title_tag else '제목 없음'
    price = price_tag.text.strip() if price_tag else '가격 없음'

    print(f"{i}. {title} - {price}원")

#드라이버 종료
driver.quit()



실제 랭킹 상품 200개

1. (신선집중) 달콤한 허니듀 멜론 대사이즈 1.8kg 2과 총 3.6kg 내외 (7/31일부터 예약발송) - 17,480원
2. 갈비탕 700g x 5팩(실온보관) - 16,390원
3. (신선집중) 닥터안스에그 햇달걀 60구 /중란 무항생제 계란 - 14,510원
4. 나이키 남녀공용 야구/골프 모자 블랙 화이트 913011 헤리티지86 892651 레거시91 1+1할인판매 - 12,710원
5. 라인바싸 탄산음료 500ml PET 3종 택 2박스 (총40입) 먹는샘물로 만들어 미네랄이 살아있는 강탄산음료 - 11,780원
6. 1+1 거창 꿀 사과 5KG 대과 (총 10KG 발송) 초특가 행사중 - 31,220원
7. 엘라스틴 스칼프클리닉 두피 딥클렌징 샴푸 950ml 2개 - 12,900원
8. 필립스 알카라인 AA건전지 AAA건전지 외 4종 - 9,900원
9. (FIFA1904/푸마) 백화점의류 창고대방출 반팔/반바지/스니커즈/쪼리 6900원부터~ - 7,900원
10. 파로돈탁스 데일리 후로라이드 잇몸치약 100g x6개 (최종혜택가 18600원) - 18,600원
11. (10%+10%) 신상품런칭 테리파머 유럽형 고중량 200g 스트라이프 호텔수건 5장 선염 수건세트 수건 타월 - 18,320원
12. 어스본코리아 태엽 토이북 시리즈 선택구매 자동차/기차/소방차/트랙터/비행기/헬리콥터/무당벌레 - 26,250원
13. (김해장미) 장미 10송이 생화 꽃 농가 직접판매  데일리꽃 농장직송 김해장미 특가 - 7,900원
14. (신선집중) 22% / 정남진 장흥 농협쌀조합 산들해랑 백미 새청무 10kg 당일도정 2024년 햅쌀 - 26,840원
15. (신선집중) 제주 미니밤호박(단호박) 보우짱 4kg 로얄과(6-20과) 실중량 / 2개 구매시 1kg 추가증정 - 12,260원
16. 곱창파는청년들 삼계탕 1kg x 4팩 (실온보관) - 18,660원
17. (오늘만 하나더) 맥퀸뉴욕 워터프루프 타투 펜라이너 1+1/

In [4]:
for i, item in enumerate(valid_items, start=1):
    title_tag = item.select_one('.box__item-title')
    price_tag = item.select_one('.box__price-seller .text__value')

    title = title_tag.text.strip() if title_tag else '제목 없음'
    sel_price = price_tag.text.strip() if price_tag else '0'

    try:
        price = int(sel_price.replace(",", "").replace("원", "")) # 가격 문자열을 정수로 바꾸기
    except:
        price = 0  # 혹시 가격 파싱이 실패할 경우

    print(f"{i}. {title} - {price:,}원")  # ,로 천 단위 구분


1. (신선집중) 달콤한 허니듀 멜론 대사이즈 1.8kg 2과 총 3.6kg 내외 (7/31일부터 예약발송) - 17,480원
2. 갈비탕 700g x 5팩(실온보관) - 16,390원
3. (신선집중) 닥터안스에그 햇달걀 60구 /중란 무항생제 계란 - 14,510원
4. 나이키 남녀공용 야구/골프 모자 블랙 화이트 913011 헤리티지86 892651 레거시91 1+1할인판매 - 12,710원
5. 라인바싸 탄산음료 500ml PET 3종 택 2박스 (총40입) 먹는샘물로 만들어 미네랄이 살아있는 강탄산음료 - 11,780원
6. 1+1 거창 꿀 사과 5KG 대과 (총 10KG 발송) 초특가 행사중 - 31,220원
7. 엘라스틴 스칼프클리닉 두피 딥클렌징 샴푸 950ml 2개 - 12,900원
8. 필립스 알카라인 AA건전지 AAA건전지 외 4종 - 9,900원
9. (FIFA1904/푸마) 백화점의류 창고대방출 반팔/반바지/스니커즈/쪼리 6900원부터~ - 7,900원
10. 파로돈탁스 데일리 후로라이드 잇몸치약 100g x6개 (최종혜택가 18600원) - 18,600원
11. (10%+10%) 신상품런칭 테리파머 유럽형 고중량 200g 스트라이프 호텔수건 5장 선염 수건세트 수건 타월 - 18,320원
12. 어스본코리아 태엽 토이북 시리즈 선택구매 자동차/기차/소방차/트랙터/비행기/헬리콥터/무당벌레 - 26,250원
13. (김해장미) 장미 10송이 생화 꽃 농가 직접판매  데일리꽃 농장직송 김해장미 특가 - 7,900원
14. (신선집중) 22% / 정남진 장흥 농협쌀조합 산들해랑 백미 새청무 10kg 당일도정 2024년 햅쌀 - 26,840원
15. (신선집중) 제주 미니밤호박(단호박) 보우짱 4kg 로얄과(6-20과) 실중량 / 2개 구매시 1kg 추가증정 - 12,260원
16. 곱창파는청년들 삼계탕 1kg x 4팩 (실온보관) - 18,660원
17. (오늘만 하나더) 맥퀸뉴욕 워터프루프 타투 펜라이너 1+1/1+1+1 2컬러 /번지지않는

In [5]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Qpswutm45!',
    database='my_db',
    charset='utf8mb4'
)

cursor = conn.cursor()

insert_query = '''
    INSERT INTO gmarket_best (item_rank, title, price)
    VALUES (%s, %s, %s)
'''


for i, item in enumerate(valid_items, start=1):
    title_tag = item.select_one('.box__item-title')
    price_tag = item.select_one('.box__price-seller .text__value')

    title = title_tag.text.strip() if title_tag else '제목 없음'
    sel_price = price_tag.text.strip() if price_tag else '0'

    try:
        price = int(sel_price.replace(",", "").replace("원", ""))
    except:
        price = 0

    cursor.execute(insert_query, (i, title, price))


conn.commit()
cursor.close()
conn.close()
print("🩷 DB 저장 완료!")


🩷 DB 저장 완료!


In [6]:
import pandas as pd

# 리스트에 저장
data = []

for i, item in enumerate(valid_items, start=1):
    title_tag = item.select_one('.box__item-title')
    price_tag = item.select_one('.box__price-seller .text__value')

    title = title_tag.text.strip() if title_tag else '제목 없음'
    sel_price = price_tag.text.strip() if price_tag else '0'

    try:
        price = int(sel_price.replace(",", "").replace("원", ""))
    except:
        price = 0

    data.append({
        'rank': i,
        'title': title,
        'price': price
    })

# DataFrame으로 변환
df = pd.DataFrame(data)

# CSV로 저장 (utf-8-sig은 엑셀 호환용)
df.to_csv('gmarket_best.csv', index=False, encoding='utf-8-sig')

print("💌 CSV 저장 완료 → gmarket_best.csv")


💌 CSV 저장 완료 → gmarket_best.csv


In [7]:
import sqlite3

# SQLite DB 연결 (없으면 자동 생성됨)
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

# 테이블 생성 (이미 생성되어 있다면 생략 가능)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS gmarket_best (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item_rank INTEGER,
        title TEXT,
        price INTEGER
    )
''')

# INSERT 쿼리 (MySQL과 거의 동일)
insert_query = '''
    INSERT INTO gmarket_best (item_rank, title, price)
    VALUES (?, ?, ?)
'''

# 예시: valid_items 크롤링 결과 반복 삽입
for i, item in enumerate(valid_items, start=1):
    title_tag = item.select_one('.box__item-title')
    price_tag = item.select_one('.box__price-seller .text__value')

    title = title_tag.text.strip() if title_tag else '제목 없음'
    sel_price = price_tag.text.strip() if price_tag else '0'

    try:
        price = int(sel_price.replace(",", "").replace("원", ""))
    except:
        price = 0

    cursor.execute(insert_query, (i, title, price))

# 커밋 후 종료
conn.commit()
cursor.close()
conn.close()
print("✅ SQLite DB 저장 완료!")


✅ SQLite DB 저장 완료!
